In [1]:
import pandas as pd
df = pd.read_csv("Sales.csv")

In [2]:
import pandas as pd

unique_pizza_ids = df['pizza_name_id'].unique()

dataframes = {}

for pizza_id in unique_pizza_ids:
    df_filtered = df[df['pizza_name_id'] == pizza_id][['week_number', 'quantity', 'order_date']]
    dataframes[pizza_id] = df_filtered

In [3]:
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_percentage_error

results = {}

for j in range(0, 91):
    example_pizza_id = unique_pizza_ids[j]
    df = dataframes[example_pizza_id]
    
    # Group by 'week_number' and sum 'quantity'
    df_weekly = df.groupby('week_number').sum().reset_index()
    df_weekly.rename(columns={'week_number': 'ds', 'quantity': 'y'}, inplace=True)

    # Split the data into training (85%) and test (15%)
    split_point = int(len(df_weekly) * 0.85)
    train_df = df_weekly[:split_point]
    test_df = df_weekly[split_point:]

    # Train the SARIMA model
    # Define seasonal order (p, d, q, s): s=52 for weekly seasonality (52 weeks in a year)
    model = SARIMAX(train_df['y'], 
                    order=(1, 1, 1),  # ARIMA parameters (p, d, q)
                    seasonal_order=(1, 1, 1, 52),  # Seasonal parameters (P, D, Q, S)
                    enforce_stationarity=False, 
                    enforce_invertibility=False)
    
    sarima_result = model.fit()

    # Forecast for the test period length
    forecast_test = sarima_result.get_forecast(steps=len(test_df))
    forecast_test_mean = forecast_test.predicted_mean

    # Calculate MAPE
    mape = mean_absolute_percentage_error(test_df['y'], forecast_test_mean)

    # Predict future for the same length as the test data
    forecast_future = sarima_result.get_forecast(steps=len(test_df))
    forecast_next_week = forecast_future.predicted_mean.iloc[0]  # First week of the forecast

    # Store the results in the dictionary
    results[f'unique_pizza_ids[{j}]'] = {
        'Pizza_id': unique_pizza_ids[j],
        'MAPE': mape,
        'Next_Week_Prediction': round(forecast_next_week)
    }

# Results can be accessed via the 'results' dictionary


c:\Users\ANNIE CHRISTINA G\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
c:\Users\ANNIE CHRISTINA G\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
c:\Users\ANNIE CHRISTINA G\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting param

In [5]:
results

{'unique_pizza_ids[0]': {'Pizza_id': 'hawaiian_m',
  'MAPE': 0.7532068625818626,
  'Next_Week_Prediction': 3},
 'unique_pizza_ids[1]': {'Pizza_id': 'classic_dlx_m',
  'MAPE': 0.28598861283643895,
  'Next_Week_Prediction': 22},
 'unique_pizza_ids[2]': {'Pizza_id': 'five_cheese_l',
  'MAPE': 0.5629393294399099,
  'Next_Week_Prediction': 26},
 'unique_pizza_ids[3]': {'Pizza_id': 'ital_supr_l',
  'MAPE': 0.6178999180643918,
  'Next_Week_Prediction': 18},
 'unique_pizza_ids[4]': {'Pizza_id': 'mexicana_m',
  'MAPE': 1.007902024446142,
  'Next_Week_Prediction': 13},
 'unique_pizza_ids[5]': {'Pizza_id': 'thai_ckn_l',
  'MAPE': 0.5316376546482404,
  'Next_Week_Prediction': 27},
 'unique_pizza_ids[6]': {'Pizza_id': 'ital_supr_m',
  'MAPE': 0.5734425971345476,
  'Next_Week_Prediction': 16},
 'unique_pizza_ids[7]': {'Pizza_id': 'prsc_argla_l',
  'MAPE': 0.40393668831168833,
  'Next_Week_Prediction': 9},
 'unique_pizza_ids[8]': {'Pizza_id': 'bbq_ckn_s',
  'MAPE': 0.6113553113553114,
  'Next_Week_Pr

In [6]:
import pandas as pd


# Extract the values from the dictionary and put them into a list
values = [v for v in results.values()]

# Create the DataFrame
df = pd.DataFrame(values)

df


,Pizza_id,MAPE,Next_Week_Prediction
0,hawaiian_m,0.753207,3
1,classic_dlx_m,0.285989,22
2,five_cheese_l,0.562939,26
3,ital_supr_l,0.617900,18
4,mexicana_m,1.007902,13
...,...,...,...
86,ckn_alfredo_l,1.045139,5
87,calabrese_s,0.423810,1
88,the_greek_l,1.041667,6
89,soppressata_m,1.150694,7


In [7]:
df.isna().sum()

Pizza_id                0
MAPE                    0
Next_Week_Prediction    0
dtype: int64

In [8]:

idf = pd.read_csv('Ingre.csv')
idf

,pizza_name_id,pizza_name,pizza_ingredients,Items_Qty_In_Grams
0,bbq_ckn_l,The Barbecue Chicken Pizza,Barbecued Chicken,40.0
1,bbq_ckn_l,The Barbecue Chicken Pizza,Red Peppers,15.0
2,bbq_ckn_l,The Barbecue Chicken Pizza,Green Peppers,20.0
3,bbq_ckn_l,The Barbecue Chicken Pizza,Tomatoes,30.0
4,bbq_ckn_l,The Barbecue Chicken Pizza,Red Onions,60.0
...,...,...,...,...
513,veggie_veg_s,The Vegetables + Vegetables Pizza,Green Peppers,20.0
514,veggie_veg_s,The Vegetables + Vegetables Pizza,Red Onions,20.0
515,veggie_veg_s,The Vegetables + Vegetables Pizza,Zucchini,10.0
516,veggie_veg_s,The Vegetables + Vegetables Pizza,Spinach,15.0


In [9]:
unique_ingredients = idf['pizza_ingredients'].unique()

ingredients_dict = {ingredient: 0 for ingredient in unique_ingredients}

(ingredients_dict)

{'Barbecued Chicken': 0,
 'Red Peppers': 0,
 'Green Peppers': 0,
 'Tomatoes': 0,
 'Red Onions': 0,
 'Barbecue Sauce': 0,
 'Bacon': 0,
 'Pepperoni': 0,
 'Italian Sausage': 0,
 'Chorizo Sausage': 0,
 'Brie Carre Cheese': 0,
 'Prosciutto': 0,
 'Caramelized Onions': 0,
 'Pears': 0,
 'Thyme': 0,
 'Garlic': 0,
 '?duja Salami': 0,
 'Pancetta': 0,
 'Friggitello Peppers': 0,
 'Chicken': 0,
 'Artichokes': 0,
 'Spinach': 0,
 'Jalapeno Peppers': 0,
 'Fontina Cheese': 0,
 'Gouda Cheese': 0,
 'Mushrooms': 0,
 'Asiago Cheese': 0,
 'Alfredo Sauce': 0,
 'Pesto Sauce': 0,
 'Mozzarella Cheese': 0,
 'Provolone Cheese': 0,
 'Smoked Gouda Cheese': 0,
 'Romano Cheese': 0,
 'Blue Cheese': 0,
 'Ricotta Cheese': 0,
 'Gorgonzola Piccante Cheese': 0,
 'Parmigiano Reggiano Cheese': 0,
 'Green Olives': 0,
 'Feta Cheese': 0,
 'Sliced Ham': 0,
 'Pineapple': 0,
 'Capocollo': 0,
 'Goat Cheese': 0,
 'Oregano': 0,
 'Calabrese Salami': 0,
 'Eggplant': 0,
 'Zucchini': 0,
 'Kalamata Olives': 0,
 'Sun-dried Tomatoes': 0,
 'P

In [10]:
df1uni = df['Pizza_id'].unique()

In [11]:
for i in range(0,45):
    filtered_rows = idf[idf['pizza_name_id'] == df1uni[i]]
    prediction = df.loc[df['Pizza_id'] ==df1uni[i], 'Next_Week_Prediction'].values[0]
    filtered_df = idf[idf['pizza_name_id'] == df1uni[i]].copy()
    filtered_df['Total_Qty_Needed'] = filtered_df['Items_Qty_In_Grams'] * prediction
    for index, row in filtered_df.iterrows():
        ingredient = row['pizza_ingredients']
        total_qty_needed = row['Total_Qty_Needed']
        if ingredient in ingredients_dict:
            ingredients_dict[ingredient] = total_qty_needed
        else:
            print(000)

In [12]:
for i in range(45,91):
    filtered_rows = idf[idf['pizza_name_id'] == df1uni[i]]
    prediction = df.loc[df['Pizza_id'] ==df1uni[i], 'Next_Week_Prediction'].values[0]
    filtered_df = idf[idf['pizza_name_id'] == df1uni[i]].copy()
    filtered_df['Total_Qty_Needed'] = filtered_df['Items_Qty_In_Grams'] * prediction
    for index, row in filtered_df.iterrows():
        ingredient = row['pizza_ingredients']
        total_qty_needed = row['Total_Qty_Needed']
        if ingredient in ingredients_dict:
            ingredients_dict[ingredient] = total_qty_needed
        else:
            print(000)

In [13]:
df_ingredients = pd.DataFrame(list(ingredients_dict.items()), columns=['Ingredient', 'Total_Qty_Needed'])

print(df_ingredients)

                 Ingredient  Total_Qty_Needed
0         Barbecued Chicken             540.0
1               Red Peppers              75.0
2             Green Peppers              75.0
3                  Tomatoes             150.0
4                Red Onions             300.0
..                      ...               ...
59                   Onions             150.0
60       Soppressata Salami             700.0
61        Peperoncini verdi              50.0
62  Thai Sweet Chilli Sauce             220.0
63         Beef Chuck Roast             600.0

[64 rows x 2 columns]


In [14]:
df_ingredients.to_excel('final_list4sarimax.xlsx',index =False)